# Introduction

This is a brief introduction to the [refractivesqlite package](https://github.com/HugoGuillen/refractiveindex.info-sqlite) developed by [Hugo Guillén](https://github.com/HugoGuillen). This package is a wrapper in Python 3 for the database of optical constants [refractiveindex.info](http://refractiveindex.info/) developed by [Mikhail Polyanskiy](https://github.com/polyanskiy). 

For better visualization, this tutorial can be downloaded as an interactive Jupyter Notebook [here](https://github.com/HugoGuillen/refractiveindex.info-sqlite/blob/master/Tutorial.ipynb).

## Features
The principal functionality of this package is to allow offline/programmatically queries to the database such as:
- Create a local SQLite database from the refractiveindex yml folder.
- Create a local SQLite database from the refractiveindex.zip url.
- Search the local database pages by approximate or exact terms.
- Search material data (refractiveindex, extinctioncoefficient) by intervals.
- Execute custom SQL queries on the database.
- Export material data (refractiveindex, extinctioncoefficient) to numpy arrays or csv files.
- Get data (refractiveindex, extinctioncoefficient) at specified wavelengths.

## Scheme
The package creates a database with the [following scheme](https://github.com/HugoGuillen/refractiveindex.info-sqlite/blob/master/ER.PNG).

![Scheme](ER.PNG "Scheme")

The table ``pages`` contains information about the materials; ``refractiveindex``  and ``extcoeff`` are self explanatory.

# Features
## Create the database

Note: once created, you don't need to create the DB again!
In both methods you can specify the interpolation_points for the case when *n* is expressed as a formula and *k* is not defined.

### From url
You can use the default database, or specify the link of the version you want to load (See 2.1.3). 

*Note: you may see some LOG warnings, that's perfectly normal.*

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.create_database_from_url()

### Check URL version

In [3]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.check_url_version()

Database file found at refractive.db
http://refractiveindex.info/download/database/rii-database-2016-01-31.zip


### Create from a custom URL 
You can find the .zip file of a version [here](http://refractiveindex.info/download.php). For example, to load the version 2016-05-25:

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.create_database_from_url(riiurl="http://refractiveindex.info/download/database/rii-database-2016-05-25.zip")

### From local yml folder

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
ymlpath = "database"
db = DB.Database(dbpath)
db.create_database_from_folder(ymlpath,interpolation_points=200)

### Load existing database

In [4]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)

Database file found at refractive.db


## Searches

### List all pages

In [ ]:
db.search_pages()

### Search pages by term

In [6]:
db.search_pages("otanicar")

3 results found.
pageid	shelf	book	page	filepath	hasrefractive	hasextinction	rangeMin	rangeMax	points
467	organic	ethylene_glycol	Sani_and_Otanicar	organic\C2H6O2 - ethylene glycol\Sani_and_Otanicar.yml	1	1	0.185	2.8	66
470	organic	propylene_glycol	Otanicar	organic\C3H8O2 - propylene glycol\Otanicar.yml	1	1	0.434	0.656	66
1561	other	TherminolVP-1	Otanicar	heat transfer fluids\Therminol VP-1\Otanicar.yml	0	1	0.2	1.5	66


### Search pages by exact term

In [7]:
db.search_pages("au",exact=True)

15 results found.
pageid	shelf	book	page	filepath	hasrefractive	hasextinction	rangeMin	rangeMax	points
27	main	Au	Rakic	main\Au\Rakic.yml	1	1	0.2066	12.4	200
28	main	Au	Olmon-ev	main\Au\Olmon-ev.yml	1	1	0.3	24.93	448
29	main	Au	Olmon-sc	main\Au\Olmon-sc.yml	1	1	0.3	24.93	448
30	main	Au	Olmon-ts	main\Au\Olmon-ts.yml	1	1	0.3	24.93	448
31	main	Au	McPeak	main\Au\McPeak.yml	1	1	0.3	1.7	141
32	main	Au	Babar	main\Au\Babar.yml	1	1	0.2066	12.4	69
33	main	Au	Ordal	main\Au\Ordal.yml	1	1	0.667	286.0	52
34	main	Au	Windt	main\Au\Windt.yml	1	1	0.00236	0.12157	36
35	main	Au	Hagemann	main\Au\Hagemann.yml	1	1	8.266e-06	248.0	149
36	main	Au	Hagemann-2	main\Au\Hagemann-2.yml	1	1	0.003542	0.8266	124
37	main	Au	Johnson	main\Au\Johnson.yml	1	1	0.1879	1.937	49
38	main	Au	Lemarchand	main\Au\Lemarchand-3.96nm.yml	1	1	0.35	1.8	291
39	main	Au	Lemarchand	main\Au\Lemarchand-4.62nm.yml	1	1	0.35	1.8	291
40	main	Au	Lemarchand	main\Au\Lemarchand-5.77nm.yml	1	1	0.35	1.8	291
41	main	Au	Lemarchand	main\Au\Lemarchand-11.7n

### Search by PageId

In [8]:
db.search_id(1542)

pageid	shelf	book	page	filepath	hasrefractive	hasextinction	rangeMin	rangeMax	points
1542	other	MLC-9200-000	Li-o	liquid crystals\MLC-9200-000\Li-o.yml	1	0	0.45	0.656	100


### Search materials by refractive index (*n*) interval

In [37]:
db.search_n(n=0.3,delta_n=.0001)

*Search n = 0.3 delta_n = 0.0001
12 results found.
pageid|shelf|book|page|wavelength|n
(4, 'main', 'Ag', 'Hagemann', 0.3351, 0.3)
(41, 'main', 'Au', 'Lemarchand', 0.705, 0.30001489)
(102, 'main', 'Cu', 'Johnson', 0.6168, 0.3)
(102, 'main', 'Cu', 'Johnson', 0.892, 0.3)
(199, 'main', 'Nb', 'Golovashkin-4.2', 2.5, 0.3)
(296, 'main', 'Pb', 'Golovashkin-4.2', 1.6, 0.3)
(1475, 'other', 'Au-Ag', 'Rioux-Au70Ag30', 0.608, 0.29996)
(1475, 'other', 'Au-Ag', 'Rioux-Au70Ag30', 0.676, 0.29992)
(1477, 'other', 'Au-Ag', 'Rioux-Au50Ag50', 0.573, 0.30004)
(1477, 'other', 'Au-Ag', 'Rioux-Au50Ag50', 0.637, 0.29994)
(1478, 'other', 'Au-Ag', 'Rioux-Au40Ag60', 0.71, 0.29996)
(1480, 'other', 'Au-Ag', 'Rioux-Au20Ag80', 0.928, 0.2999)


### Search materials by extinction coefficient (*k*) interval

In [39]:
db.search_k(k=0.3,delta_k=.0001)

*Search k = 0.3 delta_k = 0.0001
7 results found.
pageid|shelf|book|page|wavelength|k
(4, 'main', 'Ag', 'Hagemann', 0.0253, 0.3)
(74, 'main', 'C', 'Djurisic-e', 0.057251, 0.30008)
(228, 'main', 'H2O', 'Warren', 3.195, 0.3)
(251, 'main', 'Ta2O5', 'Bright-amorphous', 10.3093, 0.300047)
(323, 'main', 'CdS', 'Treharne', 0.46592328, 0.29994)
(418, 'main', 'Tm', 'Vidal-Dasilva', 0.032843493, 0.30004735)
(1584, '3d', 'crystals', 'ice', 3.195, 0.3)


### Search materials by *n,k* interval

In [40]:
db.search_nk(n=0.3, delta_n=0.1,k=0.3,delta_k=0.1)

*Search n = 0.3 delta_n = 0.1 k = 0.3 delta_k = 0.1
42 results found.
pageid|shelf|book|page|wavelength|n|k
(73, 'main', 'C', 'Djurisic-o', 0.042069, 0.399493, 0.262777)
(73, 'main', 'C', 'Djurisic-o', 0.042126, 0.397854, 0.265747)
(73, 'main', 'C', 'Djurisic-o', 0.042183, 0.396244, 0.26875)
(73, 'main', 'C', 'Djurisic-o', 0.04224, 0.394662, 0.271782)
(73, 'main', 'C', 'Djurisic-o', 0.042298, 0.39311, 0.274845)
(73, 'main', 'C', 'Djurisic-o', 0.042355, 0.391588, 0.277937)
(73, 'main', 'C', 'Djurisic-o', 0.042413, 0.390097, 0.281058)
(73, 'main', 'C', 'Djurisic-o', 0.042471, 0.388638, 0.284207)
(73, 'main', 'C', 'Djurisic-o', 0.042529, 0.387211, 0.287382)
(73, 'main', 'C', 'Djurisic-o', 0.042588, 0.385817, 0.290584)
(73, 'main', 'C', 'Djurisic-o', 0.042646, 0.384457, 0.293811)
(73, 'main', 'C', 'Djurisic-o', 0.042705, 0.383131, 0.297063)
(73, 'main', 'C', 'Djurisic-o', 0.042763, 0.38184, 0.300338)
(73, 'main', 'C', 'Djurisic-o', 0.042822, 0.380584, 0.303635)
(73, 'main', 'C', 'Djurisic-

### Custom SQL search
You can search for custom sql queries (at your own risk). The function prints the number of results, and returns a list with n-tuples ordered by the fields on the select statement. Check the Section 1.2 for the scheme.

In [49]:
#Get any page in main/Ag/ that its name contains a letter 'k'.
db.search_custom('select * from pages where shelf="main" and book="Ag" '+
                 'and page LIKE "%k%"')

2 results found.


[(0, 'main', 'Ag', 'Rakic', 'main\\Ag\\Rakic.yml', 1, 1, 0.2066, 12.4, 200),
 (1, 'main', 'Ag', 'McPeak', 'main\\Ag\\McPeak.yml', 1, 1, 0.3, 1.7, 141)]

In [54]:
#Get the wavelength and extinction coefficient from the page with id 1 
#in the interval [0.3, 0.4].
db.search_custom('''select wave,coeff 
                    from extcoeff 
                    where pageid = 1 and wave between 0.3 and 0.4''')

11 results found.


[(0.3, 0.972336834),
 (0.31, 0.572501372),
 (0.32, 0.331538655),
 (0.33, 0.585441289),
 (0.34, 1.052042671),
 (0.35, 1.326866905),
 (0.36, 1.533000885),
 (0.37, 1.708000982),
 (0.38, 1.858729622),
 (0.39, 1.99546181),
 (0.4, 2.122943979)]

In [57]:
#Get the wavelength and refractive index from the page with id 1 
#in the interval [0.3, 0.4].
db.search_custom('''select wave,refindex 
                    from refractiveindex 
                    where pageid = 1 and wave between 0.3 and 0.4''')

11 results found.


[(0.3, 1.646857286),
 (0.31, 1.455629699),
 (0.32, 0.920058628),
 (0.33, 0.233797996),
 (0.34, 0.103893124),
 (0.35, 0.07599858),
 (0.36, 0.063180003),
 (0.37, 0.056279242),
 (0.38, 0.050752406),
 (0.39, 0.048660415),
 (0.4, 0.04572895)]

In [52]:
#Get the path of the yml file, wavelength, refractive index and extinction coefficient 
#of all the pages with info on wavelength = 0.301
db.search_custom('''select p.filepath, r.wave,refindex,coeff 
                    from refractiveindex r inner join extcoeff e on r.pageid = e.pageid 
                    and r.wave = e.wave inner join pages p on r.pageid = p.pageid
                    where r.wave = .301''')

23 results found.


[('main\\Co\\Johnson.yml', 0.301, 1.44, 2.25),
 ('main\\Cr\\Johnson.yml', 0.301, 1.53, 2.34),
 ('main\\Fe\\Johnson.yml', 0.301, 1.67, 2.0),
 ('main\\Mn\\Johnson.yml', 0.301, 1.86, 2.19),
 ('main\\Ni\\Johnson.yml', 0.301, 2.02, 2.18),
 ('main\\Pd\\Johnson.yml', 0.301, 1.2, 2.29),
 ('main\\Ti\\Johnson.yml', 0.301, 1.45, 2.46),
 ('main\\V\\Johnson.yml', 0.301, 2.07, 2.95),
 ('alloys\\Au-Ag\\Rioux-Au100Ag0.yml', 0.301, 1.60958, 1.92549),
 ('alloys\\Au-Ag\\Rioux-Au90Ag10.yml', 0.301, 1.56306, 1.85315),
 ('alloys\\Au-Ag\\Rioux-Au80Ag20.yml', 0.301, 1.52131, 1.78689),
 ('alloys\\Au-Ag\\Rioux-Au70Ag30.yml', 0.301, 1.48332, 1.72633),
 ('alloys\\Au-Ag\\Rioux-Au60Ag40.yml', 0.301, 1.44868, 1.67012),
 ('alloys\\Au-Ag\\Rioux-Au50Ag50.yml', 0.301, 1.41765, 1.61551),
 ('alloys\\Au-Ag\\Rioux-Au40Ag60.yml', 0.301, 1.39132, 1.55779),
 ('alloys\\Au-Ag\\Rioux-Au30Ag70.yml', 0.301, 1.37194, 1.48921),
 ('alloys\\Au-Ag\\Rioux-Au20Ag80.yml', 0.301, 1.36413, 1.39552),
 ('alloys\\Au-Ag\\Rioux-Au10Ag90.yml', 0.3

## Get data in a numpy.array from a PageId

### Get material refractive index

In [24]:
db.get_material_n_numpy(5)

Material main\Ag\Johnson.yml loaded.


array([[ 0.1879,  1.07  ],
       [ 0.1916,  1.1   ],
       [ 0.1953,  1.12  ],
       [ 0.1993,  1.14  ],
       [ 0.2033,  1.15  ],
       [ 0.2073,  1.18  ],
       [ 0.2119,  1.2   ],
       [ 0.2164,  1.22  ],
       [ 0.2214,  1.25  ],
       [ 0.2262,  1.26  ],
       [ 0.2313,  1.28  ],
       [ 0.2371,  1.28  ],
       [ 0.2426,  1.3   ],
       [ 0.249 ,  1.31  ],
       [ 0.2551,  1.33  ],
       [ 0.2616,  1.35  ],
       [ 0.2689,  1.38  ],
       [ 0.2761,  1.41  ],
       [ 0.2844,  1.41  ],
       [ 0.2924,  1.39  ],
       [ 0.3009,  1.34  ],
       [ 0.3107,  1.13  ],
       [ 0.3204,  0.81  ],
       [ 0.3315,  0.17  ],
       [ 0.3425,  0.14  ],
       [ 0.3542,  0.1   ],
       [ 0.3679,  0.07  ],
       [ 0.3815,  0.05  ],
       [ 0.3974,  0.05  ],
       [ 0.4133,  0.05  ],
       [ 0.4305,  0.04  ],
       [ 0.4509,  0.04  ],
       [ 0.4714,  0.05  ],
       [ 0.4959,  0.05  ],
       [ 0.5209,  0.05  ],
       [ 0.5486,  0.06  ],
       [ 0.5821,  0.05  ],
 

### Get material extinction coefficients

In [25]:
db.get_material_k_numpy(5)

Material main\Ag\Johnson.yml loaded.


array([[  0.1879,   1.212 ],
       [  0.1916,   1.232 ],
       [  0.1953,   1.255 ],
       [  0.1993,   1.277 ],
       [  0.2033,   1.296 ],
       [  0.2073,   1.312 ],
       [  0.2119,   1.325 ],
       [  0.2164,   1.336 ],
       [  0.2214,   1.342 ],
       [  0.2262,   1.344 ],
       [  0.2313,   1.357 ],
       [  0.2371,   1.367 ],
       [  0.2426,   1.378 ],
       [  0.249 ,   1.389 ],
       [  0.2551,   1.393 ],
       [  0.2616,   1.387 ],
       [  0.2689,   1.372 ],
       [  0.2761,   1.331 ],
       [  0.2844,   1.264 ],
       [  0.2924,   1.161 ],
       [  0.3009,   0.964 ],
       [  0.3107,   0.616 ],
       [  0.3204,   0.392 ],
       [  0.3315,   0.829 ],
       [  0.3425,   1.142 ],
       [  0.3542,   1.419 ],
       [  0.3679,   1.657 ],
       [  0.3815,   1.864 ],
       [  0.3974,   2.07  ],
       [  0.4133,   2.275 ],
       [  0.4305,   2.462 ],
       [  0.4509,   2.657 ],
       [  0.4714,   2.869 ],
       [  0.4959,   3.093 ],
       [  0.52

### Get material data as CSV

In [26]:
db.get_material_csv(5)

Material main\Ag\Johnson.yml loaded.
Wrote 5,main,Ag,Johnson(nk).csv


### Get ALL CSVs *(Be careful)*

In [ ]:
db.get_material_csv_all(outputfolder="all")

## Interact with materials
You need to *load* a material to interact with it. Once loaded, you can make any number of operations with it.

### Load material by Id

In [27]:
mat = db.get_material(5)

Material main\Ag\Johnson.yml loaded.


### Get material info

In [28]:
print(mat.get_page_info())

OrderedDict([('pageid', 5), ('shelf', 'main'), ('book', 'Ag'), ('page', 'Johnson'), ('filepath', 'main\\Ag\\Johnson.yml'), ('hasrefractive', 1), ('hasextinction', 1), ('rangeMin', 0.1879), ('rangeMax', 1.937), ('points', 49)])


### Get all material refractive index data

In [29]:
import numpy as np
n = mat.get_complete_refractive()
print(n)

[[0.1879, 1.07], [0.1916, 1.1], [0.1953, 1.12], [0.1993, 1.14], [0.2033, 1.15], [0.2073, 1.18], [0.2119, 1.2], [0.2164, 1.22], [0.2214, 1.25], [0.2262, 1.26], [0.2313, 1.28], [0.2371, 1.28], [0.2426, 1.3], [0.249, 1.31], [0.2551, 1.33], [0.2616, 1.35], [0.2689, 1.38], [0.2761, 1.41], [0.2844, 1.41], [0.2924, 1.39], [0.3009, 1.34], [0.3107, 1.13], [0.3204, 0.81], [0.3315, 0.17], [0.3425, 0.14], [0.3542, 0.1], [0.3679, 0.07], [0.3815, 0.05], [0.3974, 0.05], [0.4133, 0.05], [0.4305, 0.04], [0.4509, 0.04], [0.4714, 0.05], [0.4959, 0.05], [0.5209, 0.05], [0.5486, 0.06], [0.5821, 0.05], [0.6168, 0.06], [0.6595, 0.05], [0.7045, 0.04], [0.756, 0.03], [0.8211, 0.04], [0.892, 0.04], [0.984, 0.04], [1.088, 0.04], [1.216, 0.09], [1.393, 0.13], [1.61, 0.15], [1.937, 0.24]]


### Get all material refractive index data as a numpy array

In [30]:
import numpy as np
n = np.array(mat.get_complete_refractive())
print(n)

[[ 0.1879  1.07  ]
 [ 0.1916  1.1   ]
 [ 0.1953  1.12  ]
 [ 0.1993  1.14  ]
 [ 0.2033  1.15  ]
 [ 0.2073  1.18  ]
 [ 0.2119  1.2   ]
 [ 0.2164  1.22  ]
 [ 0.2214  1.25  ]
 [ 0.2262  1.26  ]
 [ 0.2313  1.28  ]
 [ 0.2371  1.28  ]
 [ 0.2426  1.3   ]
 [ 0.249   1.31  ]
 [ 0.2551  1.33  ]
 [ 0.2616  1.35  ]
 [ 0.2689  1.38  ]
 [ 0.2761  1.41  ]
 [ 0.2844  1.41  ]
 [ 0.2924  1.39  ]
 [ 0.3009  1.34  ]
 [ 0.3107  1.13  ]
 [ 0.3204  0.81  ]
 [ 0.3315  0.17  ]
 [ 0.3425  0.14  ]
 [ 0.3542  0.1   ]
 [ 0.3679  0.07  ]
 [ 0.3815  0.05  ]
 [ 0.3974  0.05  ]
 [ 0.4133  0.05  ]
 [ 0.4305  0.04  ]
 [ 0.4509  0.04  ]
 [ 0.4714  0.05  ]
 [ 0.4959  0.05  ]
 [ 0.5209  0.05  ]
 [ 0.5486  0.06  ]
 [ 0.5821  0.05  ]
 [ 0.6168  0.06  ]
 [ 0.6595  0.05  ]
 [ 0.7045  0.04  ]
 [ 0.756   0.03  ]
 [ 0.8211  0.04  ]
 [ 0.892   0.04  ]
 [ 0.984   0.04  ]
 [ 1.088   0.04  ]
 [ 1.216   0.09  ]
 [ 1.393   0.13  ]
 [ 1.61    0.15  ]
 [ 1.937   0.24  ]]


### Get all material extinction coefficient data

In [31]:
k = mat.get_complete_extinction()
print(k)

[[0.1879, 1.212], [0.1916, 1.232], [0.1953, 1.255], [0.1993, 1.277], [0.2033, 1.296], [0.2073, 1.312], [0.2119, 1.325], [0.2164, 1.336], [0.2214, 1.342], [0.2262, 1.344], [0.2313, 1.357], [0.2371, 1.367], [0.2426, 1.378], [0.249, 1.389], [0.2551, 1.393], [0.2616, 1.387], [0.2689, 1.372], [0.2761, 1.331], [0.2844, 1.264], [0.2924, 1.161], [0.3009, 0.964], [0.3107, 0.616], [0.3204, 0.392], [0.3315, 0.829], [0.3425, 1.142], [0.3542, 1.419], [0.3679, 1.657], [0.3815, 1.864], [0.3974, 2.07], [0.4133, 2.275], [0.4305, 2.462], [0.4509, 2.657], [0.4714, 2.869], [0.4959, 3.093], [0.5209, 3.324], [0.5486, 3.586], [0.5821, 3.858], [0.6168, 4.152], [0.6595, 4.483], [0.7045, 4.838], [0.756, 5.242], [0.8211, 5.727], [0.892, 6.312], [0.984, 6.992], [1.088, 7.795], [1.216, 8.828], [1.393, 10.1], [1.61, 11.85], [1.937, 14.08]]


### Get all material extinction coefficient data as a numpy array

In [32]:
import numpy as np
k = np.array(mat.get_complete_extinction())
print(k)

[[  0.1879   1.212 ]
 [  0.1916   1.232 ]
 [  0.1953   1.255 ]
 [  0.1993   1.277 ]
 [  0.2033   1.296 ]
 [  0.2073   1.312 ]
 [  0.2119   1.325 ]
 [  0.2164   1.336 ]
 [  0.2214   1.342 ]
 [  0.2262   1.344 ]
 [  0.2313   1.357 ]
 [  0.2371   1.367 ]
 [  0.2426   1.378 ]
 [  0.249    1.389 ]
 [  0.2551   1.393 ]
 [  0.2616   1.387 ]
 [  0.2689   1.372 ]
 [  0.2761   1.331 ]
 [  0.2844   1.264 ]
 [  0.2924   1.161 ]
 [  0.3009   0.964 ]
 [  0.3107   0.616 ]
 [  0.3204   0.392 ]
 [  0.3315   0.829 ]
 [  0.3425   1.142 ]
 [  0.3542   1.419 ]
 [  0.3679   1.657 ]
 [  0.3815   1.864 ]
 [  0.3974   2.07  ]
 [  0.4133   2.275 ]
 [  0.4305   2.462 ]
 [  0.4509   2.657 ]
 [  0.4714   2.869 ]
 [  0.4959   3.093 ]
 [  0.5209   3.324 ]
 [  0.5486   3.586 ]
 [  0.5821   3.858 ]
 [  0.6168   4.152 ]
 [  0.6595   4.483 ]
 [  0.7045   4.838 ]
 [  0.756    5.242 ]
 [  0.8211   5.727 ]
 [  0.892    6.312 ]
 [  0.984    6.992 ]
 [  1.088    7.795 ]
 [  1.216    8.828 ]
 [  1.393   10.1   ]
 [  1.61    1

### Get material data as CSV

In [33]:
mat.to_csv(output="mat1.csv")

Wrote mat1(nk).csv


###  Get refractive index at a specified wavelength

In [34]:
mat.get_refractiveindex(451)

array(0.04004878048780487)

### Get extinction coefficient at a specified wavelength

In [35]:
mat.get_extinctioncoefficient(451)

array(2.658034146341463)